In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import matplotlib as plt
import torch
from torch.autograd import Variable


In [2]:
fil="C:/Users/Zenodia/Rscripts/data/"
df=pd.read_csv(fil+"tsIE.csv", sep=";")
df.head()
df['Month of Year']=df['Month of Year'].astype("str")
df['month']=df['Month of Year'].apply(lambda x: x[4:])
df['year']=df['Month of Year'].apply(lambda x: x[:4])
df.head()

,Month of Year,Sessions,month,year
0,200901,66953,01,2009
1,200902,61591,02,2009
2,200903,87241,03,2009
3,200904,85263,04,2009
4,200905,100813,05,2009


In [3]:
#construct cuts to make the symbols
cuts=[df['Sessions'].quantile(.1*i) for i in range(0,10)]
print (cuts , df['Sessions'].quantile(.90))


[52735.0, 69856.2, 77730.8, 87020.1, 94290.20000000001, 100113.0, 108201.00000000003, 124658.0, 136022.2, 151519.7] 151519.7


In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
# convert Session data to strings as symbols 
symbols=[]
for idx in df.index:
    if df.iloc[idx,1]< cuts[0]:
        #using normalizeString to get rid of unicode u'xx 
        symbols.append(normalizeString('a'))
    elif (df.iloc[idx,1]>=cuts[0] and df.iloc[idx,1] < cuts[1]):
        symbols.append(normalizeString('b'))
    elif (df.iloc[idx,1]>=cuts[1] and df.iloc[idx,1] < cuts[2]):
        symbols.append(normalizeString('c'))
    elif (df.iloc[idx,1]>=cuts[2] and df.iloc[idx,1] < cuts[3]):
        symbols.append(normalizeString('d'))
    elif (df.iloc[idx,1]>=cuts[3] and df.iloc[idx,1] < cuts[4]):
        symbols.append(normalizeString('e'))
    elif (df.iloc[idx,1]>=cuts[4] and df.iloc[idx,1] < cuts[5]):
        symbols.append(normalizeString('f'))
    elif (df.iloc[idx,1]>=cuts[5] and df.iloc[idx,1] < cuts[6]):
        symbols.append(normalizeString('g'))
    elif (df.iloc[idx,1]>=cuts[6] and df.iloc[idx,1] < cuts[7]):
        symbols.append(normalizeString('h'))
    elif (df.iloc[idx,1]>=cuts[7] and df.iloc[idx,1] < cuts[8]):
        symbols.append(normalizeString('i'))
    elif (df.iloc[idx,1]>=cuts[8] and df.iloc[idx,1] < cuts[9]):
        symbols.append(normalizeString('j'))
    else :
        symbols.append(normalizeString('k'))
print(len(symbols),symbols)

84 ['b', 'b', 'e', 'd', 'g', 'g', 'h', 'f', 'c', 'b', 'b', 'b', 'c', 'c', 'f', 'k', 'j', 'g', 'h', 'h', 'd', 'c', 'b', 'd', 'e', 'e', 'f', 'f', 'i', 'i', 'i', 'j', 'e', 'd', 'c', 'b', 'g', 'e', 'g', 'h', 'h', 'i', 'j', 'j', 'e', 'd', 'b', 'b', 'h', 'g', 'i', 'h', 'i', 'j', 'k', 'k', 'e', 'f', 'c', 'e', 'g', 'd', 'f', 'i', 'i', 'k', 'k', 'j', 'd', 'e', 'c', 'c', 'j', 'h', 'k', 'h', 'j', 'k', 'k', 'k', 'g', 'f', 'd', 'f']


In [6]:
df['symbols']=symbols
print ( df.year.unique())

lookup_symbols=dict(zip(df['Sessions'],df['symbols']))
lookup_symbols.items()
df.head()

['2009' '2010' '2011' '2012' '2013' '2014' '2015']


,Month of Year,Sessions,month,year,symbols
0,200901,66953,01,2009,b
1,200902,61591,02,2009,b
2,200903,87241,03,2009,e
3,200904,85263,04,2009,d
4,200905,100813,05,2009,g


In [7]:
train_data=[]

yrs=df['year'].unique().tolist()
for yr in yrs:
    
    temp=[lookup_symbols[x] for x in df.loc[(df.year==str(yr))]['Sessions'].values.tolist()]
    train_data.append(temp)

print(train_data,len(train_data))

[['b', 'b', 'e', 'd', 'g', 'g', 'h', 'f', 'c', 'b', 'b', 'b'], ['c', 'c', 'f', 'k', 'j', 'g', 'h', 'h', 'd', 'c', 'b', 'd'], ['e', 'e', 'f', 'f', 'i', 'i', 'i', 'j', 'e', 'd', 'c', 'b'], ['g', 'e', 'g', 'h', 'h', 'i', 'j', 'j', 'e', 'd', 'b', 'b'], ['h', 'g', 'i', 'h', 'i', 'j', 'k', 'k', 'e', 'f', 'c', 'e'], ['g', 'd', 'f', 'i', 'i', 'k', 'k', 'j', 'd', 'e', 'c', 'c'], ['j', 'h', 'k', 'h', 'j', 'k', 'k', 'k', 'g', 'f', 'd', 'f']] 7


In [8]:
word_to_ix = {}
for sent in train_data:
    
    for word in sent:
        if str(word) not in word_to_ix:
            word_to_ix[str(word)] = len(word_to_ix)
print(word_to_ix)

{'f': 5, 'd': 2, 'h': 4, 'j': 8, 'g': 3, 'b': 0, 'e': 1, 'c': 6, 'k': 7, 'i': 9}


In [ ]:
target_data=[]

yrs=df['year'].unique().tolist()[1:]
for yr in yrs:
    
    temp=[lookup_symbols[x] for x in df.loc[(df.year==str(yr))]['Sessions'].values.tolist()]
    target_data.append(temp)

print(target_data,len(target_data))

In [9]:
print(max([len(x) for x in train_data]))
voc_size=len(word_to_ix)
voc_size

12


10

In [ ]:
len(train_data)

In [40]:
################################ SET HYPER PARAMS

class HParams():
    def __init__(self):
        self.n_layers = 2
        self.hidden_size = 7
        self.fc_size = 11
        self.dropout = 0.9
        self.batch_size = 1
        self.lr = 0.01
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 5.
        self.cuda = False
        self.num_epoch = 500
        self.max_length = 12

hp = HParams()


In [ ]:
tryin=[word_to_ix[x] for x  in list(train_data[0])] 
input_seq= Variable(torch.FloatTensor(tryin).unsqueeze(1))
input_seq

In [11]:
class RNN_Encoder(nn.Module):
    def __init__(self, voc_size, hidden_size, n_layers=2):
        super(RNN_Encoder, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(voc_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, dropout=hp.dropout)

    def forward(self, inputs, lengths, hidden_cell=None, batch_size=hp.batch_size):
        #print("forward inputs",inputs.size()) #[12, 3])
        #print("lengths", len(lengths), type(lengths))
        if hidden_cell is None:
            if hp.cuda:
                hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda())
                cell = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda())
            else:
                hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))
                #print("forward hidde1n", hidden) #2x20x7
                cell = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))
                #print("forward cell", cell) # 2x20x7
            hidden_cell = (hidden, cell)
            #print("forward hidden2", hidden)# 2x20x7
        embedded = self.embedding(inputs.long()) # (hp.max_length*batch*emb)
        #print("forward embedded", embedded.size())#[10, 20, 7]
        input_pack = nn.utils.rnn.pack_padded_sequence(embedded, lengths)
        #print("forward input_pack", input_pack)
        output_pack, hidden_cell = self.lstm(input_pack, hidden_cell)#102x7 batch_sizes=[20, 20, 18, 15, 12, 9, 7, 1]
        #print("forward output_pack, hidden cell", output_pack, hidden_cell) #102x7, 2x20x7
        output = nn.utils.rnn.pad_packed_sequence(output_pack)
        return output_pack, hidden_cell

In [12]:
RNN_in=RNN_Encoder(voc_size, hp.hidden_size)

In [ ]:
out, h=RNN_in(input_seq,[1])
print("out",out)
print("h",h)


In [33]:

class RNN_Decoder(nn.Module):
    def __init__(self, hidden_size, fc_size, voc_size, n_layers=2):
        super(RNN_Decoder, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(voc_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, dropout=hp.dropout)
        self.fc1 = nn.Linear(hidden_size, fc_size)
        self.fc2 = nn.Linear(fc_size, voc_size)
        self.softmax = nn.LogSoftmax(dim=0)

    def forward(self, input, hidden_cell):
        # input = 1(length)*batch
        #print("decode input", input)
        #print("decode hidden_cell1", hidden_cell)
        embedded = self.embedding(input)
        #print("decoder embedded", embedded)
        output, hidden_cell = self.lstm(embedded, hidden_cell)
        #print("decoder output1", output)
        #print("decode hidden_cell2", hidden_cell)
        # output = 1(length)*batch*emb
        output = self.fc1(output.squeeze())
        #print("decoder output2", output)
        output = self.fc2(output)
        #print("decoder output3", output)
        #print("output",output.size())
        output = self.softmax(output)
        #print("decoder output4", output)
        #print("output after softmax",output.size())
        return output, hidden_cell

In [37]:
RNN_out=RNN_Decoder(hp.hidden_size, hp.fc_size, voc_size,n_layers=hp.n_layers)

In [ ]:
out=[word_to_ix[x] for x  in list(train_data[1])] 
t= Variable(torch.LongTensor(out).unsqueeze(1))

decode_out,_=RNN_out(t,h)
print("decoder out",decode_out.view(-1,voc_size)) 
print("target",t.long().view(-1))

In [42]:
encoder_optimizer = optim.Adam(RNN_in.parameters(), lr=hp.lr)
decoder_optimizer = optim.Adam(RNN_out.parameters(), lr=hp.lr)
#criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()

In [17]:

def lr_decay(optimizer):
    """Decay learning rate by a factor of lr_decay"""
    for param_group in optimizer.param_groups:
        if param_group['lr']>hp.min_lr:
            param_group['lr'] *= hp.lr_decay
    return optimizer

In [ ]:
# Loss and Optimizer

# Train the Model
for epoch in range(hp.num_epoch):

    for i in range(len(train_data)-1): # len(train_data)-1 is to hold out last year
        #encoder_optimizer = lr_decay(encoder_optimizer)
        #decoder_optimizer = lr_decay(decoder_optimizer)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        tryin=[word_to_ix[x] for x  in list(train_data[i])] 
        input_seq= Variable(torch.FloatTensor(tryin).unsqueeze(1))
        output_seq, h=RNN_in(input_seq,[1])
        last_output = output_seq
        target=[word_to_ix[x] for x  in list(train_data[i+1])] 
        t= Variable(torch.LongTensor(target).unsqueeze(1))
        decode_out,_=RNN_out(t,h)
        err = criterion(decode_out.view(-1,voc_size), t.long().view(-1))
        err.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
    if epoch%10 ==0 :
        print("loss",err)

loss Variable containing:
 2.4849
[torch.FloatTensor of size 1]

loss Variable containing:
 2.4790
[torch.FloatTensor of size 1]

loss Variable containing:
 15.9489
[torch.FloatTensor of size 1]

loss Variable containing:
 2.4849
[torch.FloatTensor of size 1]

loss Variable containing:
 12.6063
[torch.FloatTensor of size 1]

loss Variable containing:
 12.4038
[torch.FloatTensor of size 1]

loss Variable containing:
 13.5131
[torch.FloatTensor of size 1]

loss Variable containing:
 3.6397
[torch.FloatTensor of size 1]

loss Variable containing:
 9.6381
[torch.FloatTensor of size 1]

loss Variable containing:
 36.6093
[torch.FloatTensor of size 1]

loss Variable containing:
 2.4849
[torch.FloatTensor of size 1]

loss Variable containing:
 19.4122
[torch.FloatTensor of size 1]

loss Variable containing:
 12.1662
[torch.FloatTensor of size 1]

loss Variable containing:
 6.9168
[torch.FloatTensor of size 1]

loss Variable containing:
 5.8301
[torch.FloatTensor of size 1]

loss Variable cont